In [19]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('Baseball.csv')
df.head(2)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,PutOuts,Assists,Errors,Salary
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,632,43,10,475.0
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,880,82,14,480.0


I guess you wanted to exclude _Leage_ column since it contains string? We can manual ofc but let's not do that. Instead we can use dtypes

In [7]:
# example what dtypes look like
df.dtypes

AtBat        int64
Hits         int64
HmRun        int64
Runs         int64
RBI          int64
Walks        int64
Years        int64
CAtBat       int64
CHits        int64
CHmRun       int64
CRuns        int64
CRBI         int64
CWalks       int64
League      object
PutOuts      int64
Assists      int64
Errors       int64
Salary     float64
dtype: object

In [8]:
# we can check them like this
df['Runs'].dtype == 'int64'

True

In [11]:
# So we loop columns and check their dtypes like so. We want to skip dtype "object"
for col in df.columns:
    if df[col].dtype == 'object':
        print('skip column:', col)
        continue
    else:
        # find outlier and do stuff
        pass

skip column: League


I'm not too clear how your equation worked in `get_outlier_minmax` , so I'm applying simpler solution for demonstration purpose. I first replaced all outlier with `NaN` which can then be easily replaced all at once with `fillna`

In [27]:
for col in df.columns:
    if df[col].dtype == 'object':
        continue
    else:
        median = df[col].median()
        std = df[col].std()
        
        # this gives a tuple representing each row (False, False, True, etc)
        # where True is the row that contains outlier
        outliers = (df[col] - median).abs() > std
        # then fill the True rows with NaN
        df[col][outliers] = np.nan
        

In [32]:
df.head(10)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,PutOuts,Assists,Errors,Salary
0,315.0,81.0,7.0,NaN,38.0,39.0,NaN,3449.0,835.0,69.0,321.0,414.0,375.0,N,NaN,43.0,10.0,475.000
1,479.0,130.0,NaN,66.0,72.0,NaN,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,A,NaN,82.0,NaN,480.000
2,496.0,141.0,NaN,65.0,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN,354.0,N,200.0,11.0,3.0,500.000
3,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,N,NaN,40.0,4.0,91.500
4,NaN,NaN,4.0,74.0,51.0,35.0,NaN,NaN,1133.0,19.0,501.0,336.0,194.0,A,282.0,NaN,NaN,750.000
5,NaN,NaN,1.0,NaN,NaN,21.0,2.0,214.0,42.0,1.0,30.0,9.0,24.0,N,76.0,127.0,7.0,70.000
6,298.0,73.0,NaN,NaN,24.0,NaN,3.0,509.0,108.0,0.0,41.0,37.0,12.0,A,121.0,NaN,9.0,100.000
7,323.0,81.0,6.0,NaN,32.0,NaN,2.0,341.0,86.0,6.0,32.0,34.0,8.0,N,143.0,NaN,NaN,75.000
8,401.0,92.0,17.0,49.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,0.0,0.0,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,90.0,NaN,504.0,NaN,A,238.0,NaN,NaN,517.143


Now you just need to add this line to the code above to fill those NaN with median

`df[col].fillna(median, inplace=True)`

I'll reload the dataframe and run it fully below.

In [33]:
df = pd.read_csv('Baseball.csv')

for col in df.columns:
    if df[col].dtype == 'object':
        continue
    else:
        median = df[col].median()
        std = df[col].std()
        outliers = (df[col] - median).abs() > std
        df[col][outliers] = np.nan
        df[col].fillna(median, inplace=True)  # <-- added


In [34]:
df.head(10)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,PutOuts,Assists,Errors,Salary
0,315.0,81.0,7.0,52.5,38.0,39.0,6.0,3449.0,835.0,69.0,321.0,414.0,375.0,N,223.0,43.0,10.0,475.000
1,479.0,130.0,9.0,66.0,72.0,37.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,A,223.0,82.0,7.0,480.000
2,496.0,141.0,9.0,65.0,47.0,37.0,6.0,1936.0,517.5,40.0,256.0,239.5,354.0,N,200.0,11.0,3.0,500.000
3,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,N,223.0,40.0,4.0,91.500
4,414.0,103.5,4.0,74.0,51.0,35.0,6.0,1936.0,1133.0,19.0,501.0,336.0,194.0,A,282.0,45.0,7.0,750.000
5,414.0,103.5,1.0,52.5,47.0,21.0,2.0,214.0,42.0,1.0,30.0,9.0,24.0,N,76.0,127.0,7.0,70.000
6,298.0,73.0,9.0,52.5,24.0,37.0,3.0,509.0,108.0,0.0,41.0,37.0,12.0,A,121.0,45.0,9.0,100.000
7,323.0,81.0,6.0,52.5,32.0,37.0,2.0,341.0,86.0,6.0,32.0,34.0,8.0,N,143.0,45.0,7.0,75.000
8,401.0,92.0,17.0,49.0,66.0,37.0,6.0,1936.0,517.5,40.0,256.0,239.5,174.0,A,0.0,0.0,7.0,422.500
9,414.0,103.5,9.0,52.5,47.0,37.0,10.0,1936.0,517.5,90.0,256.0,504.0,174.0,A,238.0,45.0,7.0,517.143
